In [160]:
import requests
from IPython.display import Markdown
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import altair as alt
from requests.utils import quote
import os
from github import Github
from datetime import timedelta

In [161]:
fmt = "{:%Y-%m-%d}"

# Can optionally use number of days to choose dates
n_days = 60
end_date = fmt.format(pd.datetime.today())
start_date = fmt.format(pd.datetime.today() - timedelta(days=n_days))

renderer = "kaggle"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]

In [162]:
# Calculate number of days to include in plots
n_days = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days

In [163]:
# Create the renderer we want
alt.renderers.enable(renderer);

In [164]:
# Define our query object that we'll re-use for github search
class GitHubGraphQlQuery():
    def __init__(self, query):
        self.query = query
        self.headers = {"Authorization": "Bearer %s" % os.environ['GITHUB_ACCESS_TOKEN']}
        self.gql_template = """
{
  search(%s) {
    issueCount
    pageInfo {
        endCursor
        hasNextPage
    }
    nodes {
      ... on PullRequest {
        state
        id
        title
        url
        createdAt
        updatedAt
        number
        authorAssociation
        author {
          login
          url
        }
        mergedBy {
          login
          url
        }
        comments(last: 15) {
          edges {
            node {
              authorAssociation
              createdAt
              lastEditedAt
              url
              repository {
                id
              }
              author {
                login
                url
              }
            }
          }
        }
      }
      ... on Issue {
        url
        title
        state
        createdAt
        updatedAt
        number
        authorAssociation
        author {
          login
          url
        }
        comments(last: 15) {
          edges {
            node {
              author {
                login
                url
              }
              authorAssociation
              createdAt
              lastEditedAt
              repository {
                id
              }
              url
            }
          }
        }
      }
    }
  }
}
"""
    def request(self, n_pages=100, n_per_page=50):
        self.raw_data = []
        for ii in range(n_pages):
            search_query = ["first: %s" % n_per_page, 'query: "%s"' % self.query, 'type: ISSUE']
            if ii != 0:
                search_query.append('after: "%s"' % pageInfo['endCursor'])

            this_query = self.gql_template % ', '.join(search_query)
            request = requests.post('https://api.github.com/graphql', json={'query': this_query}, headers=self.headers)
            if request.status_code != 200:
                raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, this_query))
            if "errors" in request.json().keys():
                raise Exception("Query failed to run with error {}. {}".format(request.json()['errors'], this_query))
            self.request = request

            # Parse the response
            json = request.json()['data']['search']
            if ii == 0:
                print("Found {} items, which will take {} pages".format(json['issueCount'], int(np.ceil(json['issueCount'] / n_per_page))))
            self.raw_data.append(json)
            pageInfo = json['pageInfo']
            self.last_query = this_query
            if pageInfo['hasNextPage'] is False:
                break
        
        if self.raw_data[0]['issueCount'] == 0:
            print("Found no entries for query {}".format(self.query))
            self.data = None
            return
        
        # Add some extra fields
        self.data = pd.DataFrame([jj for ii in self.raw_data for jj in ii['nodes']])
        self.data['username'] = self.data['author'].map(lambda a: a['login'])
        self.data['username_url'] = self.data['author'].map(lambda a: a['url'])
        self.data['org'] = self.data['url'].map(lambda a: a.split('/')[3])
        self.data['repo'] = self.data['url'].map(lambda a: a.split('/')[4])

In [165]:
# Discourse API key
api = {'Api-Key': os.environ['DISCOURSE_API_KEY'],
       'Api-Username': os.environ['DISCOURSE_API_USERNAME']}

In [166]:
# Discourse
def topics_to_markdown(topics, n_list=10):
    body = []
    for _, topic in topics.iterrows():
        title = topic['fancy_title']
        slug = topic['slug']
        posts_count = topic['posts_count']
        url = f'https://discourse.jupyter.org/t/{slug}'
        body.append(f'* [{title}]({url}) ({posts_count} posts)')
    body = body[:n_list]
    return '\n'.join(body)

def counts_from_activity(activity):
    counts = activity.groupby('category_id').count()['bookmarked'].reset_index()
    counts['parent_category'] = None
    for ii, irow in counts.iterrows():
        if parent_categories[irow['category_id']] is not None:
            counts.loc[ii, 'parent_category'] = parent_categories[irow['category_id']]

    counts['category_id'] = counts['category_id'].map(lambda a: category_mapping[a])
    counts['parent_category'] = counts['parent_category'].map(lambda a: category_mapping[a] if a is not None else 'parent')
    is_parent = counts['parent_category'] == 'parent'
    counts.loc[is_parent, 'parent_category'] = counts.loc[is_parent, 'category_id'] 
    counts['parent/category'] = counts.apply(lambda a: a['parent_category']+'/'+a['category_id'], axis=1)
    counts = counts.sort_values(['parent_category', 'bookmarked'], ascending=False)
    return counts

# Community forum activity

The [Jupyter Community Forum](https://discourse.jupyter.org) is a place for Jovyans across the
community to talk about Jupyter tools in interactive computing and how they fit into their
workflows. It's also a place for developers to share ideas, tools, tips, and help one another.

Below are a few updates from activity in the Discourse. For more detailed information about
the activity on the Community Forum, check out these links:

* [The users page](https://discourse.jupyter.org/u) has information about user activity
* [The top posts page](https://discourse.jupyter.org/top) contains a list of top posts, sorted
  by various metrics.

In [167]:
# Get categories for IDs
url = "https://discourse.jupyter.org/site.json"
resp = requests.get(url, headers=api)
category_mapping = {cat['id']: cat['name'] for cat in resp.json()['categories']}
parent_categories = {cat['id']: cat.get("parent_category_id", None) for cat in resp.json()['categories']}

In [168]:
# Base URL to use
url = "https://discourse.jupyter.org/latest.json"

## Topics with lots of likes

"Likes" are a way for community members to say thanks for a helpful post, show their
support for an idea, or generally to share a little positivity with somebody else.
These are topics that have generated lots of likes in recent history.

In [169]:
params = {"order": "likes", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
liked = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(liked))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (98 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (40 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (28 posts)
* [JupyterHub, Binder workshop](https://discourse.jupyter.org/t/jupyterhub-binder-workshop) (14 posts)
* [Help us choose an updated &ldquo;Launch Binder&rdquo; badge!](https://discourse.jupyter.org/t/help-us-choose-an-updated-launch-binder-badge) (21 posts)
* [PyCon 2019 and mybinder.org](https://discourse.jupyter.org/t/pycon-2019-and-mybinder-org) (17 posts)
* [The Binder Federation](https://discourse.jupyter.org/t/the-binder-federation) (14 posts)
* [Jupyter community workshop: proposal on diversity, inclusion and accessibility](https://discourse.jupyter.org/t/jupyter-community-workshop-proposal-on-diversity-inclusion-and-accessibility) (20 posts)
* [Sunset the GitHub repo jupyter/help?](https://discourse.jupyter.org/t/sunset-the-github-repo-jupyter-help) (17 posts)
* [Binder/BinderHub Workshops &ndash; Help Wanted!](https://discourse.jupyter.org/t/binder-binderhub-workshops-help-wanted) (25 posts)

## Active topics on Discourse

These are topics with lots of activity in recent history.

In [170]:
params = {"order": "posts", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most posts in recent history
posts = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(posts))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (98 posts)
* [Tip: embed custom github content in a Binder link with nbgitpuller](https://discourse.jupyter.org/t/tip-embed-custom-github-content-in-a-binder-link-with-nbgitpuller) (40 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (40 posts)
* [Would a &ldquo;The Littlest Binder&rdquo; be useful?](https://discourse.jupyter.org/t/would-a-the-littlest-binder-be-useful) (31 posts)
* [Plugins of considerations](https://discourse.jupyter.org/t/plugins-of-considerations) (28 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (28 posts)
* [A proposal for JupyterHub communications](https://discourse.jupyter.org/t/a-proposal-for-jupyterhub-communications) (26 posts)
* [Binder/BinderHub Workshops &ndash; Help Wanted!](https://discourse.jupyter.org/t/binder-binderhub-workshops-help-wanted) (25 posts)
* [Rename this category to Jupyter Server?](https://discourse.jupyter.org/t/rename-this-category-to-jupyter-server) (21 posts)
* [Potential collaboration on user research?](https://discourse.jupyter.org/t/potential-collaboration-on-user-research) (21 posts)

In [171]:
counts = counts_from_activity(posts)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:            category_id  bookmarked     parent_category  \
  11       accessibility           2      Special Topics   
  9       Admin requests           1       Site Feedback   
  0                 Meta           5                Meta   
  1        announcements           1                Meta   
  6              discuss           2          JupyterHub   
  2           JupyterHub           1          JupyterHub   
  8   Enterprise Gateway           1  Enterprise Gateway   
  3       Community Chat           3      Community Chat   
  4               Binder           6              Binder   
  7              discuss           6              Binder   
  5            BinderHub           1              Binder   
  10    mybinder.org ops           1              Binder   
  
                            parent/category  
  11           Special Topics/accessibility  
  9            Site Feedback/Admin requests  
  0                               Meta/Meta  
  1                      Meta/announcements  
  6                      JupyterHub/discuss  
  2                   JupyterHub/JupyterHub  
  8   Enterprise Gateway/Enterprise Gateway  
  3           Community Chat/Community Chat  
  4                           Binder/Binder  
  7                          Binder/discuss  
  5                        Binder/BinderHub  
  10                Binder/mybinder.org ops  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'parent_category',
      type: 'nominal'
    }),
    x: X({
      field: 'parent/category',
      sort: Sort(['accessibility', 'Admin requests', 'Meta', 'announcements', 'discuss', 'JupyterHub', 'Enterprise Gateway', 'Community Chat', 'Binder', 'discuss', 'BinderHub', 'mybinder.org ops']),
      type: 'nominal'
    }),
    y: Y({
      field: 'bookmarked',
      type: 'quantitative'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## Recently-created topics

These are topics that were recently created, sorted by the amount of activity
in each one.

In [172]:
params = {"order": "created", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Sort created by the most posted for recently-created posts
created = pd.DataFrame(resp.json()['topic_list']['topics'])
created = created.sort_values('posts_count', ascending=False)
Markdown(topics_to_markdown(created))

* [Automatic HTTPS with LetsEncrypt in the context of Binder](https://discourse.jupyter.org/t/automatic-https-with-letsencrypt-in-the-context-of-binder) (12 posts)
* [Creating environment.yml for Python 2 via Binder](https://discourse.jupyter.org/t/creating-environment-yml-for-python-2-via-binder) (11 posts)
* [Why does nb2kg connect to the kernel via EG rather than directly?](https://discourse.jupyter.org/t/why-does-nb2kg-connect-to-the-kernel-via-eg-rather-than-directly) (6 posts)
* [Boilerplate needed for making a Jupyterlab extension using Vue.js](https://discourse.jupyter.org/t/boilerplate-needed-for-making-a-jupyterlab-extension-using-vue-js) (6 posts)
* [&ldquo;Astronomy Picture of the Day&rdquo; tutorial problem. Version mismatch?](https://discourse.jupyter.org/t/astronomy-picture-of-the-day-tutorial-problem-version-mismatch) (5 posts)
* [Cells not importing modules anymore](https://discourse.jupyter.org/t/cells-not-importing-modules-anymore) (5 posts)
* [HTTPS with Let&rsquo;s Encrypt on JupyterHub in Kubernetes](https://discourse.jupyter.org/t/https-with-lets-encrypt-on-jupyterhub-in-kubernetes) (4 posts)
* [Jupyter-server-proxy Gallery?](https://discourse.jupyter.org/t/jupyter-server-proxy-gallery) (4 posts)
* [Can JupyterLab access/use user installed fonts?](https://discourse.jupyter.org/t/can-jupyterlab-access-use-user-installed-fonts) (4 posts)
* [Add entries to the filebrowser contextmenu](https://discourse.jupyter.org/t/add-entries-to-the-filebrowser-contextmenu) (4 posts)

In [173]:
counts = counts_from_activity(created)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:                          category_id  bookmarked     parent_category  \
  9                                Q&A           1                 Q&A   
  13                          Notebook           1            Notebook   
  12                         nbconvert           0            Notebook   
  2                               Meta           1                Meta   
  8                         JupyterLab           3          JupyterLab   
  0   Zero to JupyterHub on Kubernetes           0          JupyterHub   
  3                         JupyterHub           0          JupyterHub   
  11                        Governance           1          Governance   
  10                Enterprise Gateway           1  Enterprise Gateway   
  4                     Community Chat           0      Community Chat   
  1                          repo help           1              Binder   
  5                             Binder           1              Binder   
  6                          BinderHub           1              Binder   
  7                            discuss           0              Binder   
  
                                  parent/category  
  9                                       Q&A/Q&A  
  13                            Notebook/Notebook  
  12                           Notebook/nbconvert  
  2                                     Meta/Meta  
  8                         JupyterLab/JupyterLab  
  0   JupyterHub/Zero to JupyterHub on Kubernetes  
  3                         JupyterHub/JupyterHub  
  11                        Governance/Governance  
  10        Enterprise Gateway/Enterprise Gateway  
  4                 Community Chat/Community Chat  
  1                              Binder/repo help  
  5                                 Binder/Binder  
  6                              Binder/BinderHub  
  7                                Binder/discuss  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'parent_category',
      type: 'nominal'
    }),
    x: X({
      field: 'parent/category',
      sort: Sort(['Q&A', 'Notebook', 'nbconvert', 'Meta', 'JupyterLab', 'Zero to JupyterHub on Kubernetes', 'JupyterHub', 'Governance', 'Enterprise Gateway', 'Community Chat', 'repo help', 'Binder', 'BinderHub', 'discuss']),
      type: 'nominal'
    }),
    y: Y({
      field: 'bookmarked',
      type: 'quantitative'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## User activity in Discourse

These are the top posters and top "like-ers" in the Discourse

In [174]:
def plot_user_data(users, column, sort=False):
    plt_data = users.sort_values(column, ascending=False).head(50)
    x = alt.X("username", sort=plt_data['username'].tolist()) if sort is True else 'username'
    ch = alt.Chart(data=plt_data).mark_bar().encode(
        x=x,
        y=column
    )
    return ch

In [175]:
url = "https://discourse.jupyter.org/directory_items.json"
params = {"period": "quarterly", "order": "post_count"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
users = pd.DataFrame(resp.json()['directory_items'])
users['username'] = users['user'].map(lambda a: a['username'])

In [176]:
plot_user_data(users.head(50), 'post_count')

Chart({
  data:     days_visited   id  likes_given  likes_received  post_count  posts_read  \
  0             79    6          212             130         164        1357   
  1             90    3          197             111         133         981   
  2             67   24           18              49          64         630   
  3             81    8           10              58          53         859   
  4             66   12            9              49          50         886   
  5             70  116           21              51          44         602   
  6             77   73           15              30          31        1538   
  7             15  220            1              24          27         189   
  8             63  239            5              10          25         467   
  9             82   30           76              17          24         323   
  10            60   99           43              19          22         534   
  11            19  234            5               3          16          84   
  12            28  485           41              17          16         240   
  15            30   60           12               9          14         338   
  14            31  372            6              13          14         209   
  13            28  419            0               3          14          77   
  16            11  254           13               1          12         175   
  17            63   11           56               9          11         775   
  18            35  432           28               6          10         272   
  19            86   77            6               7           9         822   
  20            10  510            0               6           8          10   
  21            10  390            2               1           8          60   
  22            32  217            9               5           8         177   
  23            15  145            0               8           8          67   
  29            17  227           10               2           7         259   
  33            22  333           11               4           7          28   
  32            10   71            0               4           7          76   
  31            17   34            2              14           7         146   
  30            14    9            0               9           7          93   
  25            17  462            0               1           7          73   
  28             4  679            0               8           7          62   
  26            14  351            0               8           7          60   
  24            14  487            0               0           7          83   
  27             9  566            0               1           7          29   
  37            46  222            2              13           6         128   
  39            43  303            5               3           6         162   
  38            22  281            4               8           6          42   
  40            24  134            5              15           6         226   
  36            19  237            1              25           6          52   
  35            17    4            0              17           6          42   
  34             9  195            0               1           6          33   
  41             6  562            0               0           5          22   
  42            11  247            6              14           5         183   
  43            11  430            2               3           5          41   
  44             6  556            3               6           5          65   
  45             5  565            2               3           5          33   
  46            24    7           21               5           5         223   
  47            31  184           14               2           5         260   
  48            74  229            6               0           5  

**Discourse users sorted by likes given**

In [177]:
plot_user_data(users.head(50), 'likes_given')

Chart({
  data:     days_visited   id  likes_given  likes_received  post_count  posts_read  \
  0             79    6          212             130         164        1357   
  1             90    3          197             111         133         981   
  9             82   30           76              17          24         323   
  17            63   11           56               9          11         775   
  10            60   99           43              19          22         534   
  12            28  485           41              17          16         240   
  18            35  432           28               6          10         272   
  46            24    7           21               5           5         223   
  5             70  116           21              51          44         602   
  2             67   24           18              49          64         630   
  6             77   73           15              30          31        1538   
  47            31  184           14               2           5         260   
  16            11  254           13               1          12         175   
  15            30   60           12               9          14         338   
  33            22  333           11               4           7          28   
  3             81    8           10              58          53         859   
  29            17  227           10               2           7         259   
  22            32  217            9               5           8         177   
  4             66   12            9              49          50         886   
  48            74  229            6               0           5         141   
  42            11  247            6              14           5         183   
  49            41   67            6               7           5         293   
  19            86   77            6               7           9         822   
  14            31  372            6              13          14         209   
  8             63  239            5              10          25         467   
  11            19  234            5               3          16          84   
  40            24  134            5              15           6         226   
  39            43  303            5               3           6         162   
  38            22  281            4               8           6          42   
  44             6  556            3               6           5          65   
  21            10  390            2               1           8          60   
  37            46  222            2              13           6         128   
  45             5  565            2               3           5          33   
  43            11  430            2               3           5          41   
  31            17   34            2              14           7         146   
  7             15  220            1              24          27         189   
  36            19  237            1              25           6          52   
  23            15  145            0               8           8          67   
  35            17    4            0              17           6          42   
  34             9  195            0               1           6          33   
  32            10   71            0               4           7          76   
  41             6  562            0               0           5          22   
  13            28  419            0               3          14          77   
  30            14    9            0               9           7          93   
  28             4  679            0               8           7          62   
  27             9  566            0               1           7          29   
  26            14  351            0               8           7          60   
  20            10  510            0               6           8          10   
  24            14  487            0               0           7  

**Discourse users sorted by likes given**

In [178]:
plot_user_data(users.head(50), 'likes_received')

Chart({
  data:     days_visited   id  likes_given  likes_received  post_count  posts_read  \
  0             79    6          212             130         164        1357   
  1             90    3          197             111         133         981   
  3             81    8           10              58          53         859   
  5             70  116           21              51          44         602   
  2             67   24           18              49          64         630   
  4             66   12            9              49          50         886   
  6             77   73           15              30          31        1538   
  36            19  237            1              25           6          52   
  7             15  220            1              24          27         189   
  10            60   99           43              19          22         534   
  35            17    4            0              17           6          42   
  9             82   30           76              17          24         323   
  12            28  485           41              17          16         240   
  40            24  134            5              15           6         226   
  42            11  247            6              14           5         183   
  31            17   34            2              14           7         146   
  37            46  222            2              13           6         128   
  14            31  372            6              13          14         209   
  8             63  239            5              10          25         467   
  15            30   60           12               9          14         338   
  17            63   11           56               9          11         775   
  30            14    9            0               9           7          93   
  26            14  351            0               8           7          60   
  38            22  281            4               8           6          42   
  23            15  145            0               8           8          67   
  28             4  679            0               8           7          62   
  49            41   67            6               7           5         293   
  19            86   77            6               7           9         822   
  20            10  510            0               6           8          10   
  18            35  432           28               6          10         272   
  44             6  556            3               6           5          65   
  22            32  217            9               5           8         177   
  46            24    7           21               5           5         223   
  32            10   71            0               4           7          76   
  33            22  333           11               4           7          28   
  43            11  430            2               3           5          41   
  45             5  565            2               3           5          33   
  39            43  303            5               3           6         162   
  13            28  419            0               3          14          77   
  11            19  234            5               3          16          84   
  29            17  227           10               2           7         259   
  47            31  184           14               2           5         260   
  27             9  566            0               1           7          29   
  16            11  254           13               1          12         175   
  34             9  195            0               1           6          33   
  21            10  390            2               1           8          60   
  25            17  462            0               1           7          73   
  41             6  562            0               0           5          22   
  24            14  487            0               0           7  

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [179]:
# Define colors we'll use for GitHub membership
import seaborn as sns
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_colors = sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)
author_colors = ["rgb({}, {}, {}, {})".format(*(ii*256)) for ii in author_colors]

In [180]:
from datetime import timedelta

orgs_md = []
for org in github_orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/jupyterlab](https://github.com/jupyterlab)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)
* [github.com/ipython](https://github.com/ipython)
* [github.com/binder-examples](https://github.com/binder-examples)
* [github.com/nteract](https://github.com/nteract)

In [181]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-05-14** to **2019-07-13**

In [182]:
responses = []
for org in github_orgs:
    query_prs = f"is:merged user:{org} archived:false closed:{start_date}..{end_date}"
    ghq = GitHubGraphQlQuery(query_prs)
    ghq.request()
    if ghq.data is None:
        continue
    responses.append(ghq)
    
merged = pd.concat([ii.data for ii in responses])

Found 227 items, which will take 5 pages
Found 249 items, which will take 5 pages
Found 233 items, which will take 5 pages
Found 94 items, which will take 2 pages
Found 54 items, which will take 2 pages
Found 3 items, which will take 1 pages
Found 128 items, which will take 3 pages


In [183]:
# Add a PR-specific field for closed PRs
merged['closed_by'] = merged['mergedBy'].map(lambda a: a['login'])
merged['closed_by_url'] = merged['mergedBy'].map(lambda a: a['url'])

In [184]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)

## Commentors

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [185]:
alt.Chart(data=prs_by_repo, title="Merged PRs in the last 30 days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:                 org                           repo  author
  73          nteract                       hydrogen      37
  72          nteract                      bookstore      34
  76          nteract                        nteract      25
  78          nteract                      papermill      14
  77          nteract                     nteract.io      10
  75          nteract                 minimal-plotly       2
  79          nteract                      scrapbook       2
  74          nteract                   kernel-relay       1
  80          nteract                       semiotic       1
  81          nteract                           vdom       1
  82          nteract                  vega-embed-v3       1
  57       jupyterlab                     jupyterlab     171
  61       jupyterlab                 jupyterlab-git      13
  68       jupyterlab              jupyterlab_server       9
  60       jupyterlab                jupyterlab-demo       8
  56       jupyterlab              jupyter-renderers       7
  58       jupyterlab          jupyterlab-commenting       4
  59       jupyterlab       jupyterlab-data-explorer       4
  71       jupyterlab             theme-cookiecutter       3
  55       jupyterlab      extension-cookiecutter-ts       2
  62       jupyterlab              jupyterlab-github       2
  66       jupyterlab                 jupyterlab-toc       2
  70       jupyterlab  scipy2019-jupyterlab-tutorial       2
  54       jupyterlab      extension-cookiecutter-js       1
  63       jupyterlab               jupyterlab-latex       1
  64       jupyterlab    jupyterlab-metadata-service       1
  65       jupyterlab           jupyterlab-telemetry       1
  67       jupyterlab            jupyterlab_pygments       1
  69       jupyterlab     mimerender-cookiecutter-ts       1
  43       jupyterhub            mybinder.org-deploy      92
  ..              ...                            ...     ...
  29  jupyter-widgets                      pythreejs       5
  27  jupyter-widgets             jupyterlab-sidecar       1
  28  jupyter-widgets                   midicontrols       1
  30  jupyter-widgets                     traittypes       1
  32  jupyter-widgets            widget-cookiecutter       1
  21          jupyter                       notebook      51
  19          jupyter                       nbgrader      43
  23          jupyter                    repo2docker      25
  8           jupyter             enterprise_gateway      24
  17          jupyter                      nbconvert      20
  24          jupyter                      telemetry      15
  13          jupyter                 jupyter_client      12
  10          jupyter                   jupyter-book      10
  22          jupyter                      qtconsole       9
  11          jupyter                 jupyter-sphinx       8
  15          jupyter                 jupyter_server       8
  14          jupyter                   jupyter_core       7
  12          jupyter              jupyter.github.io       5
  7           jupyter                  docker-stacks       4
  20          jupyter                       nbviewer       4
  18          jupyter                         nbdime       2
  9           jupyter                        jupyter       1
  16          jupyter                          nb2kg       1
  5           ipython                        ipython      43
  6           ipython                      traitlets       5
  3           ipython                      ipykernel       4
  4           ipython                    ipyparallel       2
  0   binder-examples                     data-quilt       1
  1   binder-examples             getting-data-xroot       1
  2   binder-examples                          latex       1
  
  [83 rows x 3 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: ['hydrogen', 'bookstore', 'nterac

### A list of merged PRs by project

In [186]:
from ipywidgets import widgets, HTML
from ipywidgets import HTML
from markdown import markdown 

In [187]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['username']
            user_url = pr['username_url']
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['closed_by']
            pr_closedby_url = pr['closed_by_url']
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

Authoring and merging stats by repo

In [188]:
# Prep our merging DF
merged_by_repo = merged.groupby(['org', 'repo', 'username']).count()['author'].reset_index().rename(columns={'author': "authored"})
closed_by_repo = merged.groupby(['org', 'repo', 'closed_by']).count()['author'].reset_index().rename(columns={'author': "closed", "closed_by": "username"})

merged_stats = pd.merge(closed_by_repo, merged_by_repo, on=['org', 'repo', 'username'], how='outer')
merged_stats = pd.melt(merged_stats, id_vars=['org', 'repo', 'username'], var_name="kind", value_name="count")
merged_stats = merged_stats.replace(np.nan, 0)

In [189]:
for ii, (iorg, idata) in enumerate(merged_stats.groupby(['org'])):
    title = f"Merges for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='count',
        color='kind',
    )
    display(ch)

Chart({
  data:                  org                repo         username      kind  count
  0    binder-examples          data-quilt          betatim    closed    1.0
  1    binder-examples  getting-data-xroot        choldgraf    closed    1.0
  2    binder-examples               latex        choldgraf    closed    1.0
  180  binder-examples          data-quilt      matheusmota    closed    0.0
  181  binder-examples  getting-data-xroot  matthewfeickert    closed    0.0
  182  binder-examples               latex        yuvipanda    closed    0.0
  462  binder-examples          data-quilt          betatim  authored    0.0
  463  binder-examples  getting-data-xroot        choldgraf  authored    0.0
  464  binder-examples               latex        choldgraf  authored    0.0
  642  binder-examples          data-quilt      matheusmota  authored    1.0
  643  binder-examples  getting-data-xroot  matthewfeickert  authored    1.0
  644  binder-examples               latex        yuvipanda  authored    1.0,
  encoding: FacetedEncoding({
    color: Color({
      field: 'kind',
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({
      field: 'count',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Merges for binder-examples in the last 60 days',
  width: 1000
})

Chart({
  data:          org         repo        username      kind  count
  3    ipython    ipykernel       blink1073    closed    1.0
  4    ipython    ipykernel           minrk    closed    3.0
  5    ipython  ipyparallel           minrk    closed    2.0
  6    ipython      ipython         Carreau    closed   43.0
  7    ipython    traitlets        rmorshea    closed    5.0
  183  ipython    ipykernel         Chilipp    closed    0.0
  184  ipython    ipykernel         hroncok    closed    0.0
  185  ipython    ipykernel        steff456    closed    0.0
  186  ipython  ipyparallel           Jamim    closed    0.0
  187  ipython  ipyparallel          ellert    closed    0.0
  188  ipython      ipython  LucianaMarques    closed    0.0
  189  ipython      ipython      OriolAbril    closed    0.0
  190  ipython      ipython       Pragnya02    closed    0.0
  191  ipython      ipython            alok    closed    0.0
  192  ipython      ipython         anntzer    closed    0.0
  193  ipython      ipython        bashtage    closed    0.0
  194  ipython      ipython         blueyed    closed    0.0
  195  ipython      ipython  brandonwillard    closed    0.0
  196  ipython      ipython          daharn    closed    0.0
  197  ipython      ipython        eobrie16    closed    0.0
  198  ipython      ipython      gooverdian    closed    0.0
  199  ipython      ipython         hroncok    closed    0.0
  200  ipython      ipython           k-zen    closed    0.0
  201  ipython      ipython          kmaork    closed    0.0
  202  ipython      ipython      pierstitus    closed    0.0
  203  ipython      ipython           rgson    closed    0.0
  204  ipython      ipython         sunpoet    closed    0.0
  205  ipython      ipython       tacaswell    closed    0.0
  206  ipython    traitlets           Jamim    closed    0.0
  207  ipython    traitlets    danielballan    closed    0.0
  ..       ...          ...             ...       ...    ...
  467  ipython  ipyparallel           minrk  authored    0.0
  468  ipython      ipython         Carreau  authored   22.0
  469  ipython    traitlets        rmorshea  authored    0.0
  645  ipython    ipykernel         Chilipp  authored    1.0
  646  ipython    ipykernel         hroncok  authored    1.0
  647  ipython    ipykernel        steff456  authored    1.0
  648  ipython  ipyparallel           Jamim  authored    1.0
  649  ipython  ipyparallel          ellert  authored    1.0
  650  ipython      ipython  LucianaMarques  authored    2.0
  651  ipython      ipython      OriolAbril  authored    2.0
  652  ipython      ipython       Pragnya02  authored    1.0
  653  ipython      ipython            alok  authored    2.0
  654  ipython      ipython         anntzer  authored    1.0
  655  ipython      ipython        bashtage  authored    1.0
  656  ipython      ipython         blueyed  authored    1.0
  657  ipython      ipython  brandonwillard  authored    1.0
  658  ipython      ipython          daharn  authored    1.0
  659  ipython      ipython        eobrie16  authored    1.0
  660  ipython      ipython      gooverdian  authored    1.0
  661  ipython      ipython         hroncok  authored    1.0
  662  ipython      ipython           k-zen  authored    1.0
  663  ipython      ipython          kmaork  authored    1.0
  664  ipython      ipython      pierstitus  authored    1.0
  665  ipython      ipython           rgson  authored    1.0
  666  ipython      ipython         sunpoet  authored    1.0
  667  ipython      ipython       tacaswell  authored    1.0
  668  ipython    traitlets           Jamim  authored    2.0
  669  ipython    traitlets    danielballan  authored    1.0
  670  ipython    traitlets   frenzymadness  authored    1.0
  671  ipython    traitlets     kevin-bates  authored    1.0
  
  [64 rows x 5 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'kind',
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({

Chart({
  data:          org                repo       username      kind  count
  8    jupyter       docker-stacks          minrk    closed    2.0
  9    jupyter       docker-stacks        parente    closed    2.0
  10   jupyter  enterprise_gateway      akchinSTC    closed    2.0
  11   jupyter  enterprise_gateway    kevin-bates    closed   16.0
  12   jupyter  enterprise_gateway       lresende    closed    6.0
  13   jupyter             jupyter        parente    closed    1.0
  14   jupyter        jupyter-book      choldgraf    closed   10.0
  15   jupyter      jupyter-sphinx  SylvainCorlay    closed    1.0
  16   jupyter      jupyter-sphinx       akhmerov    closed    6.0
  17   jupyter      jupyter-sphinx       jbweston    closed    1.0
  18   jupyter   jupyter.github.io           Ruv7    closed    1.0
  19   jupyter   jupyter.github.io         afshin    closed    1.0
  20   jupyter   jupyter.github.io      ellisonbg    closed    1.0
  21   jupyter   jupyter.github.io     jasongrout    closed    2.0
  22   jupyter      jupyter_client          MSeal    closed    1.0
  23   jupyter      jupyter_client  SylvainCorlay    closed    4.0
  24   jupyter      jupyter_client      blink1073    closed    1.0
  25   jupyter      jupyter_client          minrk    closed    1.0
  26   jupyter      jupyter_client         mpacer    closed    1.0
  27   jupyter      jupyter_client         rgbkrk    closed    4.0
  28   jupyter        jupyter_core          minrk    closed    1.0
  29   jupyter        jupyter_core         rgbkrk    closed    1.0
  30   jupyter        jupyter_core      takluyver    closed    5.0
  31   jupyter      jupyter_server  SylvainCorlay    closed    3.0
  32   jupyter      jupyter_server      blink1073    closed    1.0
  33   jupyter      jupyter_server         ivanov    closed    1.0
  34   jupyter      jupyter_server       rolweber    closed    3.0
  35   jupyter               nb2kg    kevin-bates    closed    1.0
  36   jupyter           nbconvert          MSeal    closed   15.0
  37   jupyter           nbconvert       t-makaro    closed    4.0
  ..       ...                 ...            ...       ...    ...
  727  jupyter            notebook        betatim  authored    1.0
  728  jupyter            notebook    dalanicolai  authored    1.0
  729  jupyter            notebook        ednut15  authored    3.0
  730  jupyter            notebook    fabihaahmed  authored    2.0
  731  jupyter            notebook        gnestor  authored    1.0
  732  jupyter            notebook         hdtee1  authored    1.0
  733  jupyter            notebook  joshuazeltser  authored   16.0
  734  jupyter            notebook    kevin-bates  authored    3.0
  735  jupyter            notebook         mgeier  authored    1.0
  736  jupyter            notebook        minho42  authored    1.0
  737  jupyter            notebook       mtd91429  authored    2.0
  738  jupyter            notebook       saraedum  authored    1.0
  739  jupyter            notebook     ssanderson  authored    1.0
  740  jupyter            notebook       t-makaro  authored    1.0
  741  jupyter            notebook       taohan16  authored    1.0
  742  jupyter            notebook         tholoz  authored    1.0
  743  jupyter            notebook         tmetzl  authored    1.0
  744  jupyter            notebook     vibvibgyor  authored    1.0
  745  jupyter           qtconsole          ajtam  authored    1.0
  746  jupyter           qtconsole       dalthviz  authored    2.0
  747  jupyter           qtconsole       impact27  authored    1.0
  748  jupyter         repo2docker     Xarthisius  authored    1.0
  749  jupyter         repo2docker   consideRatio  authored    1.0
  750  jupyter         repo2docker   davidanthoff  authored    1.0
  751  jupyter         repo2docker     davidrpugh  authored    1.0
  752  jupyter         repo2docker      eexwhyzee  authored    1.0
  753  jupyter         repo2docker      fmaussion  authored    1.0
  754  jupyter         repo2docke

Chart({
  data:                  org                 repo          username      kind  count
  56   jupyter-widgets           ipyleaflet     SylvainCorlay    closed    3.0
  57   jupyter-widgets           ipyleaflet       martinRenou    closed   22.0
  58   jupyter-widgets           ipywidgets     SylvainCorlay    closed   20.0
  59   jupyter-widgets           ipywidgets        jasongrout    closed    7.0
  60   jupyter-widgets           ipywidgets           vidartf    closed    3.0
  61   jupyter-widgets   jupyterlab-sidecar     SylvainCorlay    closed    1.0
  62   jupyter-widgets         midicontrols        jasongrout    closed    1.0
  63   jupyter-widgets            pythreejs           vidartf    closed    5.0
  64   jupyter-widgets           traittypes     SylvainCorlay    closed    1.0
  65   jupyter-widgets             tutorial     SylvainCorlay    closed    1.0
  66   jupyter-widgets             tutorial   maartenbreddels    closed    1.0
  67   jupyter-widgets             tutorial       martinRenou    closed    4.0
  68   jupyter-widgets             tutorial           mwcraig    closed   24.0
  69   jupyter-widgets  widget-cookiecutter           vidartf    closed    1.0
  295  jupyter-widgets           ipyleaflet         Juanlu001    closed    0.0
  296  jupyter-widgets           ipyleaflet      VasavanThiru    closed    0.0
  297  jupyter-widgets           ipyleaflet          deeplook    closed    0.0
  298  jupyter-widgets           ipyleaflet             jmp75    closed    0.0
  299  jupyter-widgets           ipyleaflet             jtpio    closed    0.0
  300  jupyter-widgets           ipyleaflet         kgiacobbi    closed    0.0
  301  jupyter-widgets           ipyleaflet          mpdwulit    closed    0.0
  302  jupyter-widgets           ipywidgets             Jamim    closed    0.0
  303  jupyter-widgets           ipywidgets         Juanlu001    closed    0.0
  304  jupyter-widgets           ipywidgets          akhmerov    closed    0.0
  305  jupyter-widgets           ipywidgets             blois    closed    0.0
  306  jupyter-widgets           ipywidgets           edolley    closed    0.0
  307  jupyter-widgets           ipywidgets            ivanov    closed    0.0
  308  jupyter-widgets           ipywidgets             jtpio    closed    0.0
  309  jupyter-widgets           ipywidgets            lheagy    closed    0.0
  310  jupyter-widgets           ipywidgets       martinRenou    closed    0.0
  ..               ...                  ...               ...       ...    ...
  529  jupyter-widgets             tutorial       martinRenou  authored   17.0
  530  jupyter-widgets             tutorial           mwcraig  authored   10.0
  531  jupyter-widgets  widget-cookiecutter           vidartf  authored    0.0
  757  jupyter-widgets           ipyleaflet         Juanlu001  authored    1.0
  758  jupyter-widgets           ipyleaflet      VasavanThiru  authored    4.0
  759  jupyter-widgets           ipyleaflet          deeplook  authored    1.0
  760  jupyter-widgets           ipyleaflet             jmp75  authored    1.0
  761  jupyter-widgets           ipyleaflet             jtpio  authored    1.0
  762  jupyter-widgets           ipyleaflet         kgiacobbi  authored    6.0
  763  jupyter-widgets           ipyleaflet          mpdwulit  authored    1.0
  764  jupyter-widgets           ipywidgets             Jamim  authored    1.0
  765  jupyter-widgets           ipywidgets         Juanlu001  authored    1.0
  766  jupyter-widgets           ipywidgets          akhmerov  authored    1.0
  767  jupyter-widgets           ipywidgets             blois  authored    1.0
  768  jupyter-widgets           ipywidgets           edolley  authored    1.0
  769  jupyter-widgets           ipywidgets            ivanov  authored    1.0
  770  jupyter-widgets           ipywidgets             jtpio  authored    1.0
  771  jupyter-widgets           ipywidgets            lheagy  authored    1.0
  772  jupyter-widgets           ipy

Chart({
  data:             org                     repo        username      kind  count
  70   jupyterhub     alabaster-jupyterhub         betatim    closed    1.0
  71   jupyterhub     alabaster-jupyterhub       choldgraf    closed    8.0
  72   jupyterhub             batchspawner      mbmilligan    closed    7.0
  73   jupyterhub                   binder         betatim    closed    1.0
  74   jupyterhub                binderhub         betatim    closed   11.0
  75   jupyterhub                binderhub       choldgraf    closed    2.0
  76   jupyterhub                binderhub    consideRatio    closed    3.0
  77   jupyterhub                binderhub           minrk    closed    1.0
  78   jupyterhub                binderhub       sgibson91    closed    2.0
  79   jupyterhub                binderhub       yuvipanda    closed    1.0
  80   jupyterhub               chartpress           minrk    closed    1.0
  81   jupyterhub               chartpress       yuvipanda    closed    1.0
  82   jupyterhub            dockerspawner           minrk    closed    1.0
  83   jupyterhub     jupyter-server-proxy         betatim    closed    1.0
  84   jupyterhub     jupyter-server-proxy       yuvipanda    closed    2.0
  85   jupyterhub               jupyterhub         betatim    closed    3.0
  86   jupyterhub               jupyterhub    consideRatio    closed    1.0
  87   jupyterhub               jupyterhub           minrk    closed    6.0
  88   jupyterhub               jupyterhub       yuvipanda    closed    3.0
  89   jupyterhub              kubespawner       yuvipanda    closed    1.0
  90   jupyterhub         ltiauthenticator       yuvipanda    closed    1.0
  91   jupyterhub      mybinder.org-deploy         betatim    closed   77.0
  92   jupyterhub      mybinder.org-deploy       choldgraf    closed    5.0
  93   jupyterhub      mybinder.org-deploy    consideRatio    closed    1.0
  94   jupyterhub      mybinder.org-deploy          henchc    closed    5.0
  95   jupyterhub      mybinder.org-deploy           minrk    closed    1.0
  96   jupyterhub      mybinder.org-deploy       sgibson91    closed    1.0
  97   jupyterhub      mybinder.org-deploy       yuvipanda    closed    2.0
  98   jupyterhub              nbgitpuller       choldgraf    closed    2.0
  99   jupyterhub              nbgitpuller       yuvipanda    closed    3.0
  ..          ...                      ...             ...       ...    ...
  814  jupyterhub           oauthenticator    DentonGentry  authored    1.0
  815  jupyterhub           oauthenticator          nifuki  authored    1.0
  816  jupyterhub           oauthenticator       sunnielyu  authored    1.0
  817  jupyterhub              sudospawner         lumbric  authored    1.0
  818  jupyterhub           systemdspawner       mortbauer  authored    1.0
  819  jupyterhub             team-compass         Zsailer  authored    2.0
  820  jupyterhub             team-compass          lheagy  authored    1.0
  821  jupyterhub             team-compass      ryanlovett  authored    1.0
  822  jupyterhub  the-littlest-jupyterhub         JuanCab  authored    1.0
  823  jupyterhub  the-littlest-jupyterhub         MayeulC  authored    1.0
  824  jupyterhub  the-littlest-jupyterhub     cornhundred  authored    1.0
  825  jupyterhub  the-littlest-jupyterhub           jtpio  authored    7.0
  826  jupyterhub  the-littlest-jupyterhub         lumbric  authored    1.0
  827  jupyterhub  the-littlest-jupyterhub            owah  authored    1.0
  828  jupyterhub  the-littlest-jupyterhub        pbugnion  authored    1.0
  829  jupyterhub  the-littlest-jupyterhub         staeiou  authored    2.0
  830  jupyterhub  the-littlest-jupyterhub        trallard  authored    1.0
  831  jupyterhub         tmpauthenticator         mohirio  authored    1.0
  832  jupyterhub            traefik-proxy  GeorgianaElena  authored    5.0
  833  jupyterhub              wrapspawner        cmd-ntrf  authored    2.0
  834  jupyterhub   zero-to-jupyte

Chart({
  data:             org                           repo         username      kind  \
  118  jupyterlab      extension-cookiecutter-js            jtpio    closed   
  119  jupyterlab      extension-cookiecutter-ts        blink1073    closed   
  120  jupyterlab              jupyter-renderers          gnestor    closed   
  121  jupyterlab              jupyter-renderers       ian-r-rose    closed   
  122  jupyterlab                     jupyterlab           afshin    closed   
  123  jupyterlab                     jupyterlab         aschlaep    closed   
  124  jupyterlab                     jupyterlab        blink1073    closed   
  125  jupyterlab                     jupyterlab        ellisonbg    closed   
  126  jupyterlab                     jupyterlab       fcollonval    closed   
  127  jupyterlab                     jupyterlab          gnestor    closed   
  128  jupyterlab                     jupyterlab       ian-r-rose    closed   
  129  jupyterlab                     jupyterlab           ivanov    closed   
  130  jupyterlab                     jupyterlab       jasongrout    closed   
  131  jupyterlab                     jupyterlab            jtpio    closed   
  132  jupyterlab                     jupyterlab   saulshanabrook    closed   
  133  jupyterlab                     jupyterlab       telamonian    closed   
  134  jupyterlab                     jupyterlab          vidartf    closed   
  135  jupyterlab          jupyterlab-commenting           acu192    closed   
  136  jupyterlab          jupyterlab-commenting           hoo761    closed   
  137  jupyterlab       jupyterlab-data-explorer   saulshanabrook    closed   
  138  jupyterlab                jupyterlab-demo      dharmaquark    closed   
  139  jupyterlab                jupyterlab-demo       ian-r-rose    closed   
  140  jupyterlab                jupyterlab-demo       jasongrout    closed   
  141  jupyterlab                 jupyterlab-git            dleen    closed   
  142  jupyterlab                 jupyterlab-git       jaipreet-s    closed   
  143  jupyterlab                 jupyterlab-git   saulshanabrook    closed   
  144  jupyterlab              jupyterlab-github       ian-r-rose    closed   
  145  jupyterlab               jupyterlab-latex       ian-r-rose    closed   
  146  jupyterlab    jupyterlab-metadata-service           acu192    closed   
  147  jupyterlab           jupyterlab-telemetry          Zsailer    closed   
  ..          ...                            ...              ...       ...   
  874  jupyterlab                     jupyterlab      parmentelat  authored   
  875  jupyterlab                     jupyterlab       rahulpshah  authored   
  876  jupyterlab                     jupyterlab          reevejd  authored   
  877  jupyterlab                     jupyterlab         tobiasjj  authored   
  878  jupyterlab                     jupyterlab          tslaton  authored   
  879  jupyterlab                     jupyterlab        yuvipanda  authored   
  880  jupyterlab                     jupyterlab          zerline  authored   
  881  jupyterlab          jupyterlab-commenting       dependabot  authored   
  882  jupyterlab       jupyterlab-data-explorer       telamonian  authored   
  883  jupyterlab                jupyterlab-demo    SylvainCorlay  authored   
  884  jupyterlab                jupyterlab-demo        TheBear44  authored   
  885  jupyterlab                 jupyterlab-git    danielgruesso  authored   
  886  jupyterlab                 jupyterlab-git       dependabot  authored   
  887  jupyterlab                 jupyterlab-git       fcollonval  authored   
  888  jupyterlab                 jupyterlab-git     neelamgehlot  authored   
  889  jupyterlab                 jupyterlab-git        urihoenig  authored   
  890  jupyterlab              jupyterlab-github       dependabot  authored   
  891  jupyterlab               jupyterlab-latex       dependabot  authored   
  892  jupyterlab           jupyterl

Chart({
  data:          org            repo        username      kind  count
  156  nteract       bookstore           MSeal    closed    3.0
  157  nteract       bookstore          mpacer    closed    7.0
  158  nteract       bookstore          rgbkrk    closed    5.0
  159  nteract       bookstore        willingc    closed   19.0
  160  nteract        hydrogen      BenRussert    closed   20.0
  161  nteract        hydrogen        aviatesk    closed    9.0
  162  nteract        hydrogen        renovate    closed    4.0
  163  nteract        hydrogen  wadethestealth    closed    4.0
  164  nteract    kernel-relay    captainsafia    closed    1.0
  165  nteract  minimal-plotly          rgbkrk    closed    2.0
  166  nteract         nteract      BenRussert    closed    8.0
  167  nteract         nteract    captainsafia    closed    6.0
  168  nteract         nteract          emeeks    closed    1.0
  169  nteract         nteract          rgbkrk    closed   10.0
  170  nteract      nteract.io        renovate    closed    9.0
  171  nteract      nteract.io          rgbkrk    closed    1.0
  172  nteract       papermill           MSeal    closed    9.0
  173  nteract       papermill          rgbkrk    closed    4.0
  174  nteract       papermill        willingc    closed    1.0
  175  nteract       scrapbook           MSeal    closed    1.0
  176  nteract       scrapbook          rgbkrk    closed    1.0
  177  nteract        semiotic          emeeks    closed    1.0
  178  nteract            vdom         gnestor    closed    1.0
  179  nteract   vega-embed-v3  CrystallineCat    closed    1.0
  442  nteract        hydrogen     JeremyStepp    closed    0.0
  443  nteract        hydrogen        hoishing    closed    0.0
  444  nteract        hydrogen       jwrussert    closed    0.0
  445  nteract    kernel-relay      BenRussert    closed    0.0
  446  nteract  minimal-plotly        renovate    closed    0.0
  447  nteract         nteract         and0111    closed    0.0
  ..       ...             ...             ...       ...    ...
  632  nteract      nteract.io        renovate  authored   10.0
  633  nteract      nteract.io          rgbkrk  authored    0.0
  634  nteract       papermill           MSeal  authored    1.0
  635  nteract       papermill          rgbkrk  authored    1.0
  636  nteract       papermill        willingc  authored    0.0
  637  nteract       scrapbook           MSeal  authored    1.0
  638  nteract       scrapbook          rgbkrk  authored    0.0
  639  nteract        semiotic          emeeks  authored    0.0
  640  nteract            vdom         gnestor  authored    1.0
  641  nteract   vega-embed-v3  CrystallineCat  authored    1.0
  904  nteract        hydrogen     JeremyStepp  authored    1.0
  905  nteract        hydrogen        hoishing  authored    1.0
  906  nteract        hydrogen       jwrussert  authored    1.0
  907  nteract    kernel-relay      BenRussert  authored    1.0
  908  nteract  minimal-plotly        renovate  authored    2.0
  909  nteract         nteract         and0111  authored    2.0
  910  nteract         nteract      dylanross3  authored    1.0
  911  nteract         nteract        renovate  authored    4.0
  912  nteract         nteract        rganardi  authored    2.0
  913  nteract         nteract  wadethestealth  authored    1.0
  914  nteract       papermill     RoelBrouwer  authored    1.0
  915  nteract       papermill  VincentBLortie  authored    1.0
  916  nteract       papermill   aabdullah-bos  authored    1.0
  917  nteract       papermill       casperdcl  authored    3.0
  918  nteract       papermill    duarteocarmo  authored    3.0
  919  nteract       papermill        khourhin  authored    1.0
  920  nteract       papermill          mwouts  authored    1.0
  921  nteract       papermill          vizhur  authored    1.0
  922  nteract       scrapbook      matthiasdv  authored    1.0
  923  nteract        semiotic        torioLuz  authored    1.0
  
  [88 rows x 5 

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [190]:
responses = []
for org in github_orgs:
    query_prs = f"is:open is:issue user:{org} archived:false created:{start_date}..{end_date}"
    ghq = GitHubGraphQlQuery(query_prs)
    ghq.request()
    if ghq.data is None:
        continue
    responses.append(ghq)
    
created = pd.concat([ii.data for ii in responses])
created['n_comments'] = created['comments'].map(lambda a: len(a['edges']))

Found 205 items, which will take 5 pages
Found 200 items, which will take 4 pages
Found 199 items, which will take 4 pages
Found 50 items, which will take 1 pages
Found 47 items, which will take 1 pages
Found 2 items, which will take 1 pages
Found 92 items, which will take 2 pages


In [191]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                           repo  number  \
  0   binder-examples                     demo-julia       1   
  1   binder-examples                   jupyter-rise       1   
  2           ipython                      ipykernel       6   
  3           ipython                    ipyparallel       3   
  4           ipython                        ipython      37   
  5           ipython               ipython-in-depth       1   
  6           jupyter                  accessibility       1   
  7           jupyter                     dashboards       1   
  8           jupyter                         design       1   
  9           jupyter                  docker-stacks      14   
  10          jupyter             enterprise_gateway       5   
  11          jupyter                        jupyter       3   
  12          jupyter                   jupyter-book      20   
  13          jupyter                  jupyter-drive       1   
  14          jupyter                 jupyter-sphinx       7   
  15          jupyter                 jupyter_client       6   
  16          jupyter                   jupyter_core       1   
  17          jupyter            jupyter_kernel_test       1   
  18          jupyter                 jupyter_server      10   
  19          jupyter                 kernel_gateway       1   
  20          jupyter                          nb2kg       1   
  21          jupyter                      nbconvert      12   
  22          jupyter                         nbdime       6   
  23          jupyter                       nbgrader      24   
  24          jupyter                       nbviewer       3   
  25          jupyter                       notebook      61   
  26          jupyter                      qtconsole       2   
  27          jupyter                    repo2docker      18   
  28          jupyter                       testpath       1   
  29  jupyter-widgets                     ipyleaflet       7   
  ..              ...                            ...     ...   
  60       jupyterlab      extension-cookiecutter-js       1   
  61       jupyterlab      extension-cookiecutter-ts       1   
  62       jupyterlab              jupyter-renderers       2   
  63       jupyterlab                     jupyterlab     147   
  64       jupyterlab            jupyterlab-celltags       5   
  65       jupyterlab          jupyterlab-commenting      12   
  66       jupyterlab       jupyterlab-data-explorer      11   
  67       jupyterlab                 jupyterlab-git       9   
  68       jupyterlab        jupyterlab-google-drive       1   
  69       jupyterlab    jupyterlab-metadata-service       1   
  70       jupyterlab           jupyterlab-telemetry       2   
  71       jupyterlab                 jupyterlab-toc       1   
  72       jupyterlab                jupyterlab_apod       1   
  73       jupyterlab                 jupyterlab_app       1   
  74       jupyterlab  scipy2018-jupyterlab-tutorial       1   
  75       jupyterlab                   team-compass       3   
  76          nteract                      bookstore      13   
  77          nteract                    coffee_boat       1   
  78          nteract                       commuter       2   
  79          nteract             commuter-on-glitch       1   
  80          nteract                             dx       3   
  81          nteract                       hydrogen      16   
  82          nteract                meeting-minutes       1   
  83          nteract                 minimal-plotly       1   
  84          nteract                        nteract      28   
  85          nteract                     nteract.io       2   
  86          nteract                      papermill      11   
  87          nteract                      scrapbook       3   
  88          nteract                       semiotic       9   
  89          nteract                           vdom       1   
  
               

In [192]:
responses = []
for org in github_orgs:
    query_prs = f"is:issue user:{org} closed:{start_date}..{end_date}"
    ghq = GitHubGraphQlQuery(query_prs)
    ghq.request()
    if ghq.data is None:
        continue
    responses.append(ghq)
    
closed = pd.concat([ii.data for ii in responses])
closed['n_comments'] = closed['comments'].map(lambda a: len(a['edges']))

Found 144 items, which will take 3 pages
Found 221 items, which will take 5 pages
Found 224 items, which will take 5 pages
Found 53 items, which will take 2 pages
Found 31 items, which will take 1 pages
Found 1 items, which will take 1 pages
Found 170 items, which will take 4 pages


In [193]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                      repo  number  \
  0   binder-examples                     bokeh       1   
  1           ipython                 ipykernel       3   
  2           ipython                   ipython      27   
  3           ipython                 traitlets       1   
  4           jupyter             atom-notebook       1   
  5           jupyter             docker-stacks       6   
  6           jupyter        enterprise_gateway      22   
  7           jupyter                   jupyter       4   
  8           jupyter              jupyter-book      11   
  9           jupyter            jupyter-sphinx      11   
  10          jupyter         jupyter.github.io       1   
  11          jupyter            jupyter_client       4   
  12          jupyter              jupyter_core       9   
  13          jupyter            jupyter_server       8   
  14          jupyter                     nb2kg       1   
  15          jupyter                 nbconvert      22   
  16          jupyter                    nbdime       1   
  17          jupyter                  nbgrader      39   
  18          jupyter                  nbviewer       4   
  19          jupyter                  notebook      52   
  20          jupyter                 qtconsole       6   
  21          jupyter               repo2docker      15   
  22          jupyter                 telemetry       3   
  23          jupyter                 terminado       1   
  24  jupyter-widgets                ipyleaflet      14   
  25  jupyter-widgets                ipywidgets      31   
  26  jupyter-widgets        jupyterlab-sidecar       1   
  27  jupyter-widgets              midicontrols       1   
  28  jupyter-widgets                 pythreejs       3   
  29  jupyter-widgets                  tutorial       3   
  ..              ...                       ...     ...   
  33       jupyterhub             dockerspawner       1   
  34       jupyterhub      jupyter-server-proxy       2   
  35       jupyterhub                jupyterhub      22   
  36       jupyterhub  jupyterhub-deploy-docker       1   
  37       jupyterhub               kubespawner       2   
  38       jupyterhub         ldapauthenticator       1   
  39       jupyterhub       mybinder.org-deploy      12   
  40       jupyterhub               nbgitpuller      10   
  41       jupyterhub            oauthenticator       1   
  42       jupyterhub              team-compass      11   
  43       jupyterhub   the-littlest-jupyterhub      48   
  44       jupyterhub          tmpauthenticator       1   
  45       jupyterhub             traefik-proxy       5   
  46       jupyterhub    zero-to-jupyterhub-k8s      15   
  47       jupyterlab         jupyter-renderers       7   
  48       jupyterlab                jupyterlab     192   
  49       jupyterlab  jupyterlab-data-explorer       7   
  50       jupyterlab           jupyterlab-demo       2   
  51       jupyterlab            jupyterlab-git      12   
  52       jupyterlab         jupyterlab-github       1   
  53       jupyterlab            jupyterlab-toc       2   
  54       jupyterlab         jupyterlab_server       1   
  55          nteract                 bookstore      11   
  56          nteract                  commuter       1   
  57          nteract                  hydrogen      97   
  58          nteract                   nteract      35   
  59          nteract                 papermill      10   
  60          nteract                 scrapbook       1   
  61          nteract                  semiotic      12   
  62          nteract                      vdom       3   
  
                                 org/repo  
  0                 binder-examples/bokeh  
  1                     ipython/ipykernel  
  2                       ipython/ipython  
  3                     ipython/traitlets  
  4                 jupyter/atom-notebook  
  5                 jupyter/docker-stacks  
  6            ju

### A list of recent issues

In [194]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(created.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for (org, repo), issues in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, issue in issues.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['username']
            user_url = issue['username_url']
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)
    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 60 days

## Commenters across repositories

In [195]:
comments = []
for items in [created, merged, closed]:
    for ii, irow in items.iterrows():
        icomments = [ii['node'] for ii in irow['comments']['edges']]
        icomments = pd.DataFrame(icomments)
        icomments['repo'] = irow['repo']
        icomments['org'] = irow['org']
        comments.append(icomments)
comments = pd.concat(comments, sort=False)
comments = comments.query("createdAt > @start_date and createdAt < @end_date")
comments['username'] = comments['author'].map(lambda a: a['login'])
comments['username_url'] = comments['author'].map(lambda a: a['url'])

In [196]:
commentors = (
    comments
    .groupby(['org', 'repo', 'username', 'authorAssociation'])
    .count()['author']
    .reset_index()
    .sort_values(['org', 'author'], ascending=False)
)

In [197]:
n_plot = 50
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='username',
        y='author',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    display(ch)

Chart({
  data:                org                repo             username authorAssociation  \
  0  binder-examples               bokeh            choldgraf            MEMBER   
  1  binder-examples               bokeh  mathematicalmichael              NONE   
  2  binder-examples          data-quilt              betatim            MEMBER   
  3  binder-examples          data-quilt          matheusmota              NONE   
  4  binder-examples          demo-julia            choldgraf            MEMBER   
  5  binder-examples  getting-data-xroot            choldgraf            MEMBER   
  6  binder-examples  getting-data-xroot      matthewfeickert       CONTRIBUTOR   
  7  binder-examples        jupyter-rise            choldgraf            MEMBER   
  8  binder-examples        jupyter-rise  mathematicalmichael              NONE   
  
     author  
  0       2  
  1       1  
  2       1  
  3       1  
  4       1  
  5       1  
  6       1  
  7       1  
  8       1  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'authorAssociation',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({
      field: 'author',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Top 50 commentors for binder-examples in the last 60 days',
  width: 1000
})

Chart({
  data:         org         repo        username authorAssociation  author
  20  ipython      ipython         Carreau            MEMBER      74
  13  ipython    ipykernel         hroncok       CONTRIBUTOR       7
  63  ipython    traitlets        rmorshea      COLLABORATOR       7
  15  ipython    ipykernel           minrk            MEMBER       5
  21  ipython      ipython  LucianaMarques       CONTRIBUTOR       5
  12  ipython    ipykernel      ccordoba12       CONTRIBUTOR       4
  36  ipython      ipython         efiring       CONTRIBUTOR       4
  60  ipython    traitlets     kevin-bates       CONTRIBUTOR       4
  50  ipython      ipython    sanketShetty              NONE       3
  9   ipython    ipykernel         Carreau            MEMBER       2
  11  ipython    ipykernel       blink1073            MEMBER       2
  18  ipython  ipyparallel           minrk            MEMBER       2
  27  ipython      ipython         blueyed       CONTRIBUTOR       2
  34  ipython      ipython          dlukes              NONE       2
  35  ipython      ipython     drunkwcodes              NONE       2
  38  ipython      ipython        fehiepsi              NONE       2
  42  ipython      ipython      jamesmyatt              NONE       2
  43  ipython      ipython        jfhbrook              NONE       2
  47  ipython      ipython          kmaork       CONTRIBUTOR       2
  49  ipython      ipython         mlucool              NONE       2
  53  ipython      ipython       tacaswell       CONTRIBUTOR       2
  55  ipython      ipython      thrasibule              NONE       2
  56  ipython      ipython         wenjuno              NONE       2
  58  ipython    traitlets           Jamim       CONTRIBUTOR       2
  62  ipython    traitlets          rgbkrk            MEMBER       2
  10  ipython    ipykernel         Chilipp       CONTRIBUTOR       1
  14  ipython    ipykernel      jamesmyatt              NONE       1
  16  ipython    ipykernel          noklam              NONE       1
  17  ipython  ipyparallel           Jamim       CONTRIBUTOR       1
  19  ipython      ipython    Benjamin-Lee              NONE       1
  22  ipython      ipython      OriolAbril       CONTRIBUTOR       1
  23  ipython      ipython        aakhmetz              NONE       1
  24  ipython      ipython            alok       CONTRIBUTOR       1
  25  ipython      ipython         anntzer       CONTRIBUTOR       1
  26  ipython      ipython        bashtage       CONTRIBUTOR       1
  28  ipython      ipython  brandonwillard       CONTRIBUTOR       1
  29  ipython      ipython    brunobeltran              NONE       1
  30  ipython      ipython      ccordoba12       CONTRIBUTOR       1
  31  ipython      ipython         csheaff              NONE       1
  32  ipython      ipython          daharn       CONTRIBUTOR       1
  33  ipython      ipython         dickmao              NONE       1
  37  ipython      ipython  farisachugthai              NONE       1
  39  ipython      ipython       gdhameeja              NONE       1
  40  ipython      ipython         hroncok       CONTRIBUTOR       1
  41  ipython      ipython      ikizhvatov              NONE       1
  44  ipython      ipython       jmansilla              NONE       1
  45  ipython      ipython  joaomarcosgris              NONE       1
  46  ipython      ipython           kernc              NONE       1
  48  ipython      ipython           kmtac              NONE       1
  51  ipython      ipython        stonebig       CONTRIBUTOR       1,
  encoding: FacetedEncoding({
    color: Color({
      field: 'authorAssociation',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      

Chart({
  data:          org                repo       username authorAssociation  author
  200  jupyter            nbgrader       jhamrick            MEMBER      98
  298  jupyter         repo2docker        betatim      COLLABORATOR      73
  276  jupyter            notebook      takluyver            MEMBER      71
  90   jupyter  enterprise_gateway    kevin-bates            MEMBER      66
  104  jupyter        jupyter-book      choldgraf            MEMBER      46
  166  jupyter           nbconvert          MSeal            MEMBER      44
  119  jupyter      jupyter-sphinx       akhmerov            MEMBER      37
  153  jupyter      jupyter_server        Zsailer            MEMBER      28
  320  jupyter         repo2docker      yuvipanda            MEMBER      26
  301  jupyter         repo2docker   consideRatio      COLLABORATOR      22
  321  jupyter           telemetry        Zsailer            MEMBER      20
  134  jupyter      jupyter_client  SylvainCorlay            MEMBER      19
  180  jupyter           nbconvert       t-makaro      COLLABORATOR      18
  324  jupyter           telemetry      yuvipanda            MEMBER      17
  204  jupyter            nbgrader        nthiery       CONTRIBUTOR      16
  208  jupyter            nbgrader        rkdarst       CONTRIBUTOR      16
  114  jupyter        jupyter-book    psychemedia              NONE      15
  286  jupyter           qtconsole     ccordoba12      COLLABORATOR      15
  152  jupyter      jupyter_server  SylvainCorlay            MEMBER      14
  159  jupyter      jupyter_server       rolweber      COLLABORATOR      14
  216  jupyter            nbviewer       krinsman       CONTRIBUTOR      14
  322  jupyter           telemetry     jaipreet-s      COLLABORATOR      14
  86   jupyter  enterprise_gateway       echarles       CONTRIBUTOR      13
  133  jupyter      jupyter_client          MSeal            MEMBER      12
  300  jupyter         repo2docker      choldgraf            MEMBER      12
  310  jupyter         repo2docker          minrk            MEMBER      12
  94   jupyter  enterprise_gateway       suryag10              NONE      11
  257  jupyter            notebook  joshuazeltser       CONTRIBUTOR      11
  264  jupyter            notebook       lresende            MEMBER      10
  274  jupyter            notebook     ssanderson       CONTRIBUTOR      10
  302  jupyter         repo2docker   davidanthoff       CONTRIBUTOR      10
  118  jupyter      jupyter-sphinx  SylvainCorlay            MEMBER       9
  150  jupyter        jupyter_core      takluyver            MEMBER       9
  207  jupyter            nbgrader    psychemedia              NONE       9
  92   jupyter  enterprise_gateway      michzimny       CONTRIBUTOR       8
  186  jupyter              nbdime        vidartf      COLLABORATOR       8
  189  jupyter            nbgrader        Sefriol              NONE       8
  275  jupyter            notebook       t-makaro       CONTRIBUTOR       8
  77   jupyter       docker-stacks        parente            MEMBER       7
  195  jupyter            nbgrader        dsblank            MEMBER       7
  196  jupyter            nbgrader     gertingold       CONTRIBUTOR       7
  87   jupyter  enterprise_gateway         esevan      COLLABORATOR       6
  122  jupyter      jupyter-sphinx   hugokerstens       CONTRIBUTOR       6
  145  jupyter      jupyter_client         rgbkrk            MEMBER       6
  218  jupyter            nbviewer        parente            MEMBER       6
  259  jupyter            notebook    kevin-bates            MEMBER       6
  303  jupyter         repo2docker     davidrpugh       CONTRIBUTOR       6
  83   jupyter  enterprise_gateway       IMAM9AIS       CONTRIBUTOR       5
  100  jupyter             jupyter        parente            MEMBER       5
  102  jupyter        jupyter-book          BENR0              NONE       5,
  encoding: FacetedEncoding({
    color: Color({
      field: 'authorAssociation',
      scale: Scale({
     

Chart({
  data:                  org                repo         username authorAssociation  \
  348  jupyter-widgets          ipywidgets    SylvainCorlay            MEMBER   
  359  jupyter-widgets          ipywidgets       jasongrout            MEMBER   
  336  jupyter-widgets          ipyleaflet      martinRenou            MEMBER   
  373  jupyter-widgets          ipywidgets          vidartf            MEMBER   
  386  jupyter-widgets           pythreejs          vidartf            MEMBER   
  393  jupyter-widgets            tutorial          mwcraig       CONTRIBUTOR   
  357  jupyter-widgets          ipywidgets            hainm              NONE   
  339  jupyter-widgets          ipyleaflet         stonebig       CONTRIBUTOR   
  392  jupyter-widgets            tutorial      martinRenou            MEMBER   
  335  jupyter-widgets          ipyleaflet        kgiacobbi       CONTRIBUTOR   
  365  jupyter-widgets          ipywidgets          mwcraig       CONTRIBUTOR   
  347  jupyter-widgets          ipywidgets        Ricyteach       CONTRIBUTOR   
  381  jupyter-widgets           pythreejs         aliddell              NONE   
  334  jupyter-widgets          ipyleaflet            jtpio       CONTRIBUTOR   
  361  jupyter-widgets          ipywidgets          kafonek              NONE   
  362  jupyter-widgets          ipywidgets  maartenbreddels            MEMBER   
  368  jupyter-widgets          ipywidgets       rskabelund              NONE   
  391  jupyter-widgets            tutorial  maartenbreddels            MEMBER   
  369  jupyter-widgets          ipywidgets       ssanderson       CONTRIBUTOR   
  377  jupyter-widgets  jupyterlab-sidecar       doutriaux1              NONE   
  384  jupyter-widgets           pythreejs         stonebig              NONE   
  330  jupyter-widgets          ipyleaflet     VasavanThiru       CONTRIBUTOR   
  331  jupyter-widgets          ipyleaflet         chbrandt              NONE   
  332  jupyter-widgets          ipyleaflet         deeplook       CONTRIBUTOR   
  333  jupyter-widgets          ipyleaflet      dwr-psandhu              NONE   
  341  jupyter-widgets          ipywidgets  DanielAristidou              NONE   
  346  jupyter-widgets          ipywidgets        Liwellyen              NONE   
  349  jupyter-widgets          ipywidgets           aa2858              NONE   
  353  jupyter-widgets          ipywidgets        beasteers              NONE   
  355  jupyter-widgets          ipywidgets        choldgraf              NONE   
  370  jupyter-widgets          ipywidgets         stonebig       CONTRIBUTOR   
  375  jupyter-widgets  jupyterlab-sidecar    SylvainCorlay            MEMBER   
  379  jupyter-widgets        midicontrols       jasongrout            MEMBER   
  394  jupyter-widgets            tutorial          vidartf            MEMBER   
  328  jupyter-widgets          ipyleaflet        Juanlu001       CONTRIBUTOR   
  329  jupyter-widgets          ipyleaflet    SylvainCorlay            MEMBER   
  337  jupyter-widgets          ipyleaflet         mpdwulit       CONTRIBUTOR   
  338  jupyter-widgets          ipyleaflet      paulathomas              NONE   
  340  jupyter-widgets          ipywidgets       DYSpider13              NONE   
  342  jupyter-widgets          ipywidgets         DevFabrx              NONE   
  343  jupyter-widgets          ipywidgets   IlyaKazakevich       CONTRIBUTOR   
  344  jupyter-widgets          ipywidgets        JohnIssue              NONE   
  345  jupyter-widgets          ipywidgets        Juanlu001       CONTRIBUTOR   
  350  jupyter-widgets          ipywidgets         akhmerov       CONTRIBUTOR   
  351  jupyter-widgets          ipywidgets      andymcarter              NONE   
  352  jupyter-widgets          ipywidgets   artur-trzesiok              NONE   
  354  jupyter-widgets          ipywidgets           bybyte              NONE   
  356  jupyter-widgets          ipywidgets         dazzag24              NONE   
  358  jupyter-wi

Chart({
  data:             org                     repo         username authorAssociation  \
  591  jupyterhub  the-littlest-jupyterhub        yuvipanda            MEMBER   
  511  jupyterhub      mybinder.org-deploy          betatim            MEMBER   
  551  jupyterhub             team-compass          betatim            MEMBER   
  552  jupyterhub             team-compass        choldgraf            MEMBER   
  406  jupyterhub                binderhub          betatim            MEMBER   
  610  jupyterhub   zero-to-jupyterhub-k8s     consideRatio            MEMBER   
  531  jupyterhub              nbgitpuller        yuvipanda            MEMBER   
  488  jupyterhub               jupyterhub          ryogesh       CONTRIBUTOR   
  520  jupyterhub      mybinder.org-deploy      mael-le-gal            MEMBER   
  513  jupyterhub      mybinder.org-deploy        choldgraf            MEMBER   
  402  jupyterhub             batchspawner          rkdarst       CONTRIBUTOR   
  567  jupyterhub  the-littlest-jupyterhub   GeorgianaElena      COLLABORATOR   
  561  jupyterhub             team-compass        sgibson91            MEMBER   
  408  jupyterhub                binderhub        choldgraf            MEMBER   
  420  jupyterhub                binderhub        sgibson91            MEMBER   
  559  jupyterhub             team-compass            minrk            MEMBER   
  523  jupyterhub      mybinder.org-deploy        yuvipanda            MEMBER   
  525  jupyterhub              nbgitpuller        choldgraf            MEMBER   
  597  jupyterhub            traefik-proxy            minrk            MEMBER   
  479  jupyterhub               jupyterhub            minrk            MEMBER   
  528  jupyterhub              nbgitpuller      parmentelat       CONTRIBUTOR   
  563  jupyterhub             team-compass        yuvipanda            MEMBER   
  618  jupyterhub   zero-to-jupyterhub-k8s           manics       CONTRIBUTOR   
  409  jupyterhub                binderhub     consideRatio            MEMBER   
  515  jupyterhub      mybinder.org-deploy           henchc            MEMBER   
  491  jupyterhub               jupyterhub        vilhelmen       CONTRIBUTOR   
  595  jupyterhub            traefik-proxy   GeorgianaElena      COLLABORATOR   
  445  jupyterhub               jupyterhub   GeorgianaElena       CONTRIBUTOR   
  449  jupyterhub               jupyterhub  RandyBetancourt              NONE   
  498  jupyterhub              kubespawner     consideRatio            MEMBER   
  616  jupyterhub   zero-to-jupyterhub-k8s         joshbode       CONTRIBUTOR   
  400  jupyterhub             batchspawner       mbmilligan            MEMBER   
  493  jupyterhub               jupyterhub        yuvipanda            MEMBER   
  521  jupyterhub      mybinder.org-deploy            minrk            MEMBER   
  540  jupyterhub           oauthenticator            minrk            MEMBER   
  549  jupyterhub             team-compass          Zsailer            MEMBER   
  558  jupyterhub             team-compass      mael-le-gal            MEMBER   
  564  jupyterhub  the-littlest-jupyterhub      CyborgDroid              NONE   
  417  jupyterhub                binderhub         memeplex              NONE   
  428  jupyterhub            dockerspawner        alvinhuff              NONE   
  460  jupyterhub               jupyterhub      brayanrodpr              NONE   
  500  jupyterhub              kubespawner           gsemet       CONTRIBUTOR   
  617  jupyterhub   zero-to-jupyterhub-k8s        lakshaykc              NONE   
  411  jupyterhub                binderhub          jhamman       CONTRIBUTOR   
  418  jupyterhub                binderhub            minrk            MEMBER   
  422  jupyterhub                binderhub        yuvipanda            MEMBER   
  440  jupyterhub     jupyter-server-proxy         quasiben       CONTRIBUTOR   
  467  jupyterhub               jupyterhub          eylenth              NONE   
  471  jupyterhub

Chart({
  data:             org                      repo             username  \
  726  jupyterlab                jupyterlab           jasongrout   
  676  jupyterlab                jupyterlab            blink1073   
  728  jupyterlab                jupyterlab  jupyterlab-dev-mode   
  720  jupyterlab                jupyterlab           ian-r-rose   
  780  jupyterlab                jupyterlab              vidartf   
  764  jupyterlab                jupyterlab       saulshanabrook   
  771  jupyterlab                jupyterlab           telamonian   
  703  jupyterlab                jupyterlab            ellisonbg   
  662  jupyterlab                jupyterlab               afshin   
  804  jupyterlab            jupyterlab-git           jaipreet-s   
  772  jupyterlab                jupyterlab            tgeorgeux   
  640  jupyterlab                jupyterlab               BoPeng   
  667  jupyterlab                jupyterlab             aschlaep   
  789  jupyterlab  jupyterlab-data-explorer       saulshanabrook   
  698  jupyterlab                jupyterlab             domoritz   
  776  jupyterlab                jupyterlab              tslaton   
  724  jupyterlab                jupyterlab              jakevdp   
  659  jupyterlab                jupyterlab              Zsailer   
  727  jupyterlab                jupyterlab                jtpio   
  701  jupyterlab                jupyterlab         duarteocarmo   
  714  jupyterlab                jupyterlab              gnestor   
  735  jupyterlab                jupyterlab           krassowski   
  782  jupyterlab                jupyterlab         vtrokhymenko   
  794  jupyterlab           jupyterlab-demo           ian-r-rose   
  795  jupyterlab           jupyterlab-demo           jasongrout   
  721  jupyterlab                jupyterlab              ibdafna   
  748  jupyterlab                jupyterlab           michaelaye   
  769  jupyterlab                jupyterlab             stonebig   
  803  jupyterlab            jupyterlab-git           fcollonval   
  812  jupyterlab            jupyterlab-git       saulshanabrook   
  709  jupyterlab                jupyterlab         flying-sheep   
  774  jupyterlab                jupyterlab            timkpaine   
  785  jupyterlab                jupyterlab            yuvipanda   
  825  jupyterlab         jupyterlab_server            blink1073   
  646  jupyterlab                jupyterlab           IgorDouven   
  674  jupyterlab                jupyterlab            bionicles   
  705  jupyterlab                jupyterlab           fcollonval   
  808  jupyterlab            jupyterlab-git  mathematicalmichael   
  675  jupyterlab                jupyterlab              bjtho08   
  693  jupyterlab                jupyterlab         davidanthoff   
  702  jupyterlab                jupyterlab             echarles   
  722  jupyterlab                jupyterlab               ivanov   
  723  jupyterlab                jupyterlab           jaipreet-s   
  725  jupyterlab                jupyterlab            jakirkham   
  806  jupyterlab            jupyterlab-git           luispuerto   
  669  jupyterlab                jupyterlab            athornton   
  673  jupyterlab                jupyterlab              betatim   
  687  jupyterlab                jupyterlab           daejong123   
  692  jupyterlab                jupyterlab     david-waterworth   
  734  jupyterlab                jupyterlab              kevmk04   
  
      authorAssociation  author  
  726            MEMBER     425  
  676            MEMBER     241  
  728              NONE     151  
  720            MEMBER     129  
  780            MEMBER     127  
  764            MEMBER      49  
  771            MEMBER      47  
  703            MEMBER      45  
  662            MEMBER      43  
  804      COLLABORATOR      24  
  772            MEMBER      20  
  640            MEMBER      19  
  667            MEMBER      17  
  789            MEMBER      17  
  698

Chart({
  data:          org            repo             username authorAssociation  author
  879  nteract        hydrogen       wadethestealth            MEMBER     135
  859  nteract        hydrogen             aviatesk            MEMBER      57
  843  nteract       bookstore               mpacer            MEMBER      53
  842  nteract       bookstore              codecov              NONE      36
  847  nteract       bookstore             willingc            MEMBER      36
  913  nteract       papermill                MSeal            MEMBER      35
  851  nteract        hydrogen           BenRussert            MEMBER      34
  924  nteract       papermill               rgbkrk            MEMBER      18
  903  nteract         nteract               rgbkrk            MEMBER      15
  906  nteract         nteract                stale              NONE      14
  846  nteract       bookstore                 todo              NONE      13
  890  nteract         nteract         captainsafia            MEMBER      13
  907  nteract         nteract       wadethestealth            MEMBER      11
  910  nteract      nteract.io                  now              NONE      10
  923  nteract       papermill               mwouts            MEMBER      10
  927  nteract       scrapbook                MSeal            MEMBER       9
  865  nteract        hydrogen           kiwi0fruit       CONTRIBUTOR       8
  884  nteract         nteract           BenRussert            MEMBER       8
  933  nteract        semiotic               emeeks            MEMBER       8
  919  nteract       papermill         duarteocarmo            MEMBER       7
  862  nteract        hydrogen             hoishing            MEMBER       6
  871  nteract        hydrogen            pepicello              NONE       6
  877  nteract        hydrogen             stevengj              NONE       6
  888  nteract         nteract         ShalokShalom              NONE       6
  926  nteract       papermill             willingc            MEMBER       6
  900  nteract         nteract      nicolaskruchten              NONE       5
  939  nteract            vdom             rmorshea            MEMBER       5
  841  nteract       bookstore                MSeal            MEMBER       4
  870  nteract        hydrogen         optionsraghu              NONE       4
  875  nteract        hydrogen             shr00mie              NONE       4
  876  nteract        hydrogen      sschuldenzucker              NONE       4
  908  nteract         nteract             willingc            MEMBER       4
  914  nteract       papermill          RoelBrouwer       CONTRIBUTOR       4
  915  nteract       papermill  avisrivastava254084              NONE       4
  929  nteract       scrapbook           matthiasdv            MEMBER       4
  844  nteract       bookstore               rgbkrk            MEMBER       3
  863  nteract        hydrogen             isajacas              NONE       3
  864  nteract        hydrogen          jesusloplar              NONE       3
  868  nteract        hydrogen             n-riesco            MEMBER       3
  883  nteract  minimal-plotly               rgbkrk            MEMBER       3
  911  nteract       papermill               0anton              NONE       3
  928  nteract       scrapbook              codecov              NONE       3
  937  nteract            vdom              gnestor            MEMBER       3
  849  nteract        commuter         captainsafia            MEMBER       2
  853  nteract        hydrogen            Mikeymice              NONE       2
  855  nteract        hydrogen                Tbosa              NONE       2
  857  nteract        hydrogen              ald0405              NONE       2
  861  nteract        hydrogen                goi42              NONE       2
  866  nteract        hydrogen           kylebarron            MEMBER       2
  872  nteract        hydrogen             renovate       CONTRIBUTOR       2,
  encod

In [198]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>